In [ ]:
from eval_1022 import *
import json
import numpy as np

# train_val_dataset = json.load(open("/home/pcarragh/dev/webqa/MultiModalQA/data/WebQA_train_val_obj_v2.json.pert.lama.v3", "r"))

train_dataset = json.load(open("/home/pcarragh/dev/webqa/MultiModalQA/data/WebQA_train_val_obj_v2.json.pert.lama.v3", "r"))
val_dataset = json.load(open("/home/pcarragh/dev/webqa/MultiModalQA/data/WebQA_train_val_obj_v2.json.pert.lama.v3.val.gpt.json", "r"))

final_dataset = {}

val_generations = 0
train_generations = 0

val_match_generations = 0
train_match_generations = 0

exp_name = "llava"

# merge train and val generated images
for k, v in val_dataset.items():
    if val_dataset[k]['split'] == 'val':
        if 'A_perturbed' not in v:
            continue
        val_generations += len(v['A_perturbed'])
        unmatched_idxs = []
        for label_idx in v['A_perturbed']:
            gen_label = v['A_perturbed'][label_idx]
            if label_idx not in v['A_perturbed_gpt']:
                unmatched_idxs.append(label_idx)
            else:
                gpt_label = v['A_perturbed_gpt'][label_idx]
                if gen_label[:3] == gpt_label[:3]:
                    val_match_generations += 1
                    final_dataset[k] = v
                else:
                    unmatched_idxs.append(label_idx)
        # remove unmatched idxs
        for idx in unmatched_idxs:
            del v['A_perturbed'][idx]
            if idx in v['A_perturbed_gpt']:
                del v['A_perturbed_gpt'][idx]

for k, v in train_dataset.items():
    if train_dataset[k]['split'] == 'train':
        if 'A_perturbed' not in v:
            continue
        train_generations += len(v['A_perturbed'])
        unmatched_idxs = []
        # fix A_perturbed_gpt idxs off by one
        v['A_perturbed_gpt'] = { str(int(k) - 1): v for k, v in v['A_perturbed_gpt'].items() }
        for label_idx in v['A_perturbed']:
            gen_label = v['A_perturbed'][label_idx]
            if label_idx not in v['A_perturbed_gpt']:
                unmatched_idxs.append(label_idx)
            else: 
                gpt_label = v['A_perturbed_gpt'][label_idx]
                if gen_label[:3] == gpt_label[:3]:
                    train_match_generations += 1
                    final_dataset[k] = v
                else:
                    unmatched_idxs.append(label_idx)
        # remove unmatched idxs
        for idx in unmatched_idxs:
            del v['A_perturbed'][idx]
            if idx in v['A_perturbed_gpt']:
                del v['A_perturbed_gpt'][idx]

print("Val generations: ", val_generations)
print("Val match generations: ", val_match_generations)
print("Train generations: ", train_generations)
print("Train match generations: ", train_match_generations)

# json.dump(final_dataset, open("/home/pcarragh/dev/webqa/MultiModalQA/data/WebQA_train_val_lama.json", "w"))

# count keys in each qcate
qcate_count = {}
for k, v in final_dataset.items():
    qcate = v['Qcate'].lower()
    if qcate not in qcate_count:
        qcate_count[qcate] = 0
    qcate_count[qcate] += 1
    
print(qcate_count)

eval_data = {k:v for k, v in final_dataset.items() if v['Qcate'].lower() in ['color']}#, 'shape', 'yesno', 'number']}

# json.dump(eval_data, open(f"WebQA_train_val_color_gpt_matched_{exp_name}.json", "w"))

print(len(eval_data))
sum([len(v['A_perturbed']) for k, v in eval_data.items()])

Val generations:  901
Val match generations:  86
Train generations:  7751
Train match generations:  890
{'color': 572, 'shape': 87}
572


In [38]:
from llava.mm_utils import get_model_name_from_path
from llava.eval.run_llava import eval_model, eval
from llava.model.builder import load_pretrained_model
from llava.utils import disable_torch_init

disable_torch_init()
model_path = "liuhaotian/llava-v1.5-7b"
model_name = get_model_name_from_path(model_path)
tokenizer, model, image_processor, context_len = load_pretrained_model(
    model_path, None, model_name
)

def llava_eval_on_webqa_sample(question, image_files, webqa_image_loader = True):
    args = type('Args', (), {
        "model_path": model_path,
        "model_base": None,
        "model_name": model_name,
        "query": question,
        "conv_mode": None,
        "image_file": image_files,
        "sep": ",",
        "temperature": 0,
        "top_p": None,
        "num_beams": 1,
        "max_new_tokens": 512,
        "webqa": webqa_image_loader,
    })()

    return eval(tokenizer, model, image_processor,  args)

def webqa_accuracy(answer, label, Qcate):
    if Qcate == 'color':
        F1_avg, F1_max, EM, RE_avg, PR_avg = compute_vqa_metrics([answer], label[0], "", color_set)
    elif Qcate == 'shape': 
        F1_avg, F1_max, EM, RE_avg, PR_avg = compute_vqa_metrics([answer], label[0], "", shape_set)
    elif Qcate == 'yesno': 
        F1_avg, F1_max, EM, RE_avg, PR_avg = compute_vqa_metrics([answer], label[0], "", yesno_set)
    elif Qcate == 'number': 
        F1_avg, F1_max, EM, RE_avg, PR_avg = compute_vqa_metrics([answer], label[0], "", {"NUMBER"})
    else:
        return None
    return (F1_avg, F1_max, EM, RE_avg, PR_avg)

def accuracy_agg_results(qa_results):
    single_image_keys = [k for k in qa_results.keys() if len(eval_data[k]['img_posFacts']) == 1]
    two_image_keys = [k for k in qa_results.keys() if len(eval_data[k]['img_posFacts']) == 2]

    single_acc = np.mean([PR_avg for key, (F1_avg, F1_max, EM, RE_avg, PR_avg) in qa_results.items() if key in single_image_keys])
    two_image_acc = np.mean([PR_avg for key, (F1_avg, F1_max, EM, RE_avg, PR_avg) in qa_results.items() if key in two_image_keys])
    avr_acc = np.mean([PR_avg for key, (F1_avg, F1_max, EM, RE_avg, PR_avg) in qa_results.items()])
    return (single_acc, two_image_acc, avr_acc)

/home/pcarragh/miniconda3/envs/llava/lib/python3.10/site-packages/huggingface_hub/file_download.py:1142: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
You are using a model of type llava to instantiate a model of type llava_llama. This is not supported for all configurations of models and can yield errors.
Loading checkpoint shards: 100%|██████████| 2/2 [00:03<00:00,  1.79s/it]


### LLAVA baseline on original images that have perturbations

In [28]:
from tqdm import tqdm

llava_results_baseline = {}
for k in tqdm(list(eval_data.keys())):
    question = eval_data[k]['Q']
    image_files = ','.join([str(img_data['image_id']) for img_data in eval_data[k]['img_posFacts']])
    try:
        answer = llava_eval_on_webqa_sample(question, image_files, True)
    except:
        answer = 'ERROR: llava failed'
    label = eval_data[k]['A']
    eval_data[k]['A_llava'] = answer
    Qcate = eval_data[k]['Qcate'].lower()
    llava_results_baseline[k] = webqa_accuracy(answer, label, Qcate)

print(accuracy_agg_results(llava_results_baseline))

  0%|          | 0/572 [00:00<?, ?it/s]/home/pcarragh/miniconda3/envs/llava/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:392: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/home/pcarragh/miniconda3/envs/llava/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:397: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `None` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
100%|██████████| 572/572 [07:49<00:00,  1.22it/s]

(0.7189327485380118, 0.6530172413793104, 0.705565268065268)


### Correct despite blank image

In [29]:
llava_results_blank = {}
for k in tqdm(list(eval_data.keys())):
    question = eval_data[k]['Q']
    image_files ='playground/counterfactual_exp/BLANK.jpg'
    try:
        answer = llava_eval_on_webqa_sample(question, image_files, False)
    except:
        answer = 'ERROR: llava failed'
    label = eval_data[k]['A']
    eval_data[k]['A_llava_blank'] = answer
    Qcate = eval_data[k]['Qcate'].lower()
    llava_results_blank[k] = webqa_accuracy(answer, label, Qcate)
print(accuracy_agg_results(llava_results_blank))

  0%|          | 0/572 [00:00<?, ?it/s]/home/pcarragh/miniconda3/envs/llava/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:392: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/home/pcarragh/miniconda3/envs/llava/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:397: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `None` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
100%|██████████| 572/572 [07:38<00:00,  1.25it/s]

(0.2978801169590643, 0.3712643678160919, 0.3127622377622377)


### Correct despite perturbation

In [42]:
llava_results_perturbed_original_label = {}
llava_results_perturbed_generated_label = {}

for k in tqdm(list(eval_data.keys())):
    llava_results_perturbed_original_label[k] = {}
    llava_results_perturbed_generated_label[k] = {}
    eval_data[k]['A_perturbed_llava'] = {}
    question = eval_data[k]['Q']
    for idx, label in eval_data[k]['A_perturbed'].items():
        image_id = eval_data[k]['img_posFacts'][0]['image_id']
        # TODO: add image 2 (image loader can't handle this now)
        # image_files = ','.join([str(img_data['image_id']) for img_data in eval_data[k]['img_posFacts']])
        image_files = f"/home/pcarragh/dev/webqa/segment/Inpaint-Anything/results/webqa/{eval_data[k]['split']}/{str(image_id)}_{k}_{idx}.jpeg"
        try:
            answer = llava_eval_on_webqa_sample(question, image_files, False)
        except:
            answer = 'ERROR: llava failed'

        original_label = eval_data[k]['A']
        eval_data[k]['A_perturbed_llava'][idx] = answer
        Qcate = eval_data[k]['Qcate'].lower()
        llava_results_perturbed_original_label[k][idx] = webqa_accuracy(answer, original_label, Qcate)
        llava_results_perturbed_generated_label[k][idx] = webqa_accuracy(answer, [label], Qcate)

100%|██████████| 572/572 [11:33<00:00,  1.21s/it]


In [45]:
def accuracy_agg_generated_results(qa_results):
    single_image_keys = [k for k in qa_results.keys() if len(eval_data[k]['img_posFacts']) == 1]
    two_image_keys = [k for k in qa_results.keys() if len(eval_data[k]['img_posFacts']) == 2]

    single_acc = np.mean([PR_avg for key, dict in qa_results.items() if key in single_image_keys for idx, (_,_,_,_,PR_avg) in dict.items()])
    two_image_acc = np.mean([PR_avg for key, dict in qa_results.items() if key in two_image_keys for idx, (_,_,_,_,PR_avg) in dict.items()])
    avr_acc = np.mean([PR_avg for key, dict in qa_results.items() for idx, (_,_,_,_,PR_avg) in dict.items()])
    
    return (single_acc, two_image_acc, avr_acc)

print(accuracy_agg_generated_results(llava_results_perturbed_original_label))
print(accuracy_agg_generated_results(llava_results_perturbed_generated_label))

(0.13203463203463203, 0.15963855421686746, 0.13736903376018625)
(0.7085137085137085, 0.6882530120481928, 0.7045983701979045)


In [52]:
import pandas as pd

baseline_accs = accuracy_agg_results(llava_results_baseline)
blank_accs = accuracy_agg_results(llava_results_blank)
perturbed_original_label_accs = accuracy_agg_generated_results(llava_results_perturbed_original_label)
perturbed_generated_label_acc = accuracy_agg_generated_results(llava_results_perturbed_generated_label)

columns = ['experiment name', 'single_image', 'two_image', 'average']
accuracy_agg_df = pd.DataFrame(columns=columns)
accuracy_agg_df['experiment name'] = ['baseline', 'blank', 'perturbed_original_label', 'perturbed_generated_label']
accuracy_agg_df['single_image'] = [baseline_accs[0], blank_accs[0], perturbed_original_label_accs[0], perturbed_generated_label_acc[0]]
accuracy_agg_df['two_image'] = [baseline_accs[1], blank_accs[1], perturbed_original_label_accs[1], perturbed_generated_label_acc[1]]
accuracy_agg_df['average'] = [baseline_accs[2], blank_accs[2], perturbed_original_label_accs[2], perturbed_generated_label_acc[2]]
accuracy_agg_df.to_csv("results/{exp_name}.csv", index=False)
accuracy_agg_df


,experiment name,single_image,two_image,average
0,baseline,0.718933,0.653017,0.705565
1,blank,0.297880,0.371264,0.312762
2,perturbed_original_label,0.132035,0.159639,0.137369
3,perturbed_generated_label,0.708514,0.688253,0.704598


### Correct despite blank image and perturbation

In [50]:
# for questions that llava gets right regardless of having a blank image, check accuracy on counterfactual examples
perturbation_ignored = {key:idx for key, dict in llava_results_perturbed_original_label.items() for idx, (_,_,_,_,acc) in dict.items() if acc > 0}
# and accuray on llava_results_perturbed_generated_label is 0
perturbation_ignored = {key:idx for key, idx in perturbation_ignored.items() if llava_results_perturbed_generated_label[key][idx][4] == 0}
blank_ignored = set([k for k, (_,_,_,_,acc) in llava_results_blank.items() if acc > 0])
parametric_ans_keys = set(perturbation_ignored.keys()).intersection(blank_ignored)
len(parametric_ans_keys)

33

In [51]:
for key in parametric_ans_keys:
    print(f"Key: {key}, Q: {eval_data[key]['Q']}")
    print(f"A: {eval_data[key]['A']}")
    print(f"A_llava: {eval_data[key]['A_llava']}")
    print(f"A_llava_blank: {eval_data[key]['A_llava_blank']}")
    idx = perturbation_ignored[key]
    print(f"A_perturbed_llava: {eval_data[key]['A_perturbed_llava'][idx]}")
    
    print(f"{idx}, A_perturbed: {eval_data[key]['A_perturbed'][idx]}")
    # , A_llava_blank: {eval_data[key]['A_llava_blank']}, A_perturbed_llava: {eval_data[key]['A_perturbed_llava']}")

Key: d5ca49d80dba11ecb1e81171463288e9, Q: "What color are the walls at The Blue Hour Contemporary Art Gallery in Vancouver?"
A: ['"The walls at The Blue Hour Contemporary Art Gallery in Vancouver are white."']
A_llava: The walls at The Blue Hour Contemporary Art Gallery in Vancouver are white.
A_llava_blank: The walls at The Blue Hour Contemporary Art Gallery in Vancouver are white.
A_perturbed_llava: The walls at The Blue Hour Contemporary Art Gallery in Vancouver are black.
2, A_perturbed: gray
Key: d5cf04aa0dba11ecb1e81171463288e9, Q: "What color is the word across the top floor of the Enbridge Centre?"
A: ['"The word across the top floor of the Enbridge Centre is white."']
A_llava: The word across the top floor of the Enbridge Centre is yellow.
A_llava_blank: The word "Enbridge" is white.
A_perturbed_llava: The word across the top floor of the Enbridge Centre is white.
0, A_perturbed: teal
Key: d5c8efca0dba11ecb1e81171463288e9, Q: "What color feathers surround more of the eye of th

: 